In [92]:
import math as mt
import pandas as pd
import numpy as np
from scipy.stats import norm

import matplotlib
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:,.2f}'.format

from scipy.stats import linregress

import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
df=pd.read_csv("D:\\Export\JWTG_AB_DuringTest_OnlyAB_IDLevelUPandOriginal.csv")

In [3]:
df.head(10)

,AB_ID,USER_ID,PRICE,PRODUCT,ISNEWACCOUNT,PLATFORM,COUNTRY
0,original,1700244631376,4.99,com.ludia.jurassicworld.specialoffer_fto_1,0,Android,United States
1,level_up,1700272188399,9.99,com.ludia.jurassicworld.memauto1,0,Android,United States
2,original,1700271842657,74.99,com.ludia.jurassicworld.cardgeneralpack75,0,Android,United States
3,original,1700273796855,19.99,com.ludia.jurassicworld.specialoffer,0,iOS,United States
4,original,1700234432367,9.99,com.ludia.jurassicworld.memauto1,0,Android,United States
5,original,1700200257436,9.99,com.ludia.jurassicworld.memauto1,0,iOS,United States
6,original,1700284859184,4.99,com.ludia.jurassicworld.specialoffer_fto_1,1,iOS,United States
7,original,1700282629149,24.99,com.ludia.jurassicworld.cardsuperrarepack,0,Android,United States
8,original,1700272373857,1.99,com.ludia.jurassicworld.hc_starter,0,iOS,United States
9,level_up,1700284078375,4.99,com.ludia.jurassicworld.hc_small,0,iOS,United States


In [8]:
df.USER_ID=df.USER_ID.apply(str)
df.ISNEWACCOUNT=df.ISNEWACCOUNT.apply(str)

In [7]:
df.describe()

,PRICE
count,"12,541.00"
mean,12.32
std,17.20
min,0.00
25%,2.99
50%,9.99
75%,9.99
max,99.99


In [9]:
print('nb unique spender:', df.USER_ID.nunique())
print('nb purchases:', df.PRICE.count())

nb unique spender: 5735
nb purchases: 12541


In [66]:
print('Number of Unique Spenders:', df['USER_ID'].nunique(), "\n" )

print('Number of Purchases',"\n",df['AB_ID'].value_counts(),"\n" )

print('AVG Amount of Purchase by Test Groups', "\n", "\n", df.groupby('AB_ID').agg({'PRICE':'mean'}),"\n" ,"\n" )

print('AVG Amount of Purchase by AccountAge', "\n", "\n", df.groupby('ISNEWACCOUNT')['PRICE'].mean(),"\n" ,"\n" )

print('AVG Amount of Purchase by Test Groups And AccountAge',"\n", "\n", df.groupby(['AB_ID','ISNEWACCOUNT'])['PRICE'].mean(),"\n","\n"  )

print('AVG Amount of Purchase by Test Groups And Platforms',"\n","\n", df.groupby(['PLATFORM', 'AB_ID'])['PRICE'].mean(),"\n","\n"  )
print('AVG Amount of Purchase by Test Groups And Platforms',"\n","\n", df.groupby(['ISNEWACCOUNT','PLATFORM', 'AB_ID'])['PRICE'].mean(),"\n","\n"  )
print('AVG Amount of Purchase by Products',"\n", "\n", df.groupby(['PLATFORM','PRODUCT','AB_ID',  ])['PRICE'].mean(),"\n","\n"  )

Number of Unique Spenders: 5735 

Number of Purchases 
 original    9941
level_up    2600
Name: AB_ID, dtype: int64 

AVG Amount of Purchase by Test Groups 
 
           PRICE
AB_ID          
level_up  11.68
original  12.49 
 

AVG Amount of Purchase by AccountAge 
 
 ISNEWACCOUNT
0   12.42
1   11.49
Name: PRICE, dtype: float64 
 

AVG Amount of Purchase by Test Groups And AccountAge 
 
 AB_ID     ISNEWACCOUNT
level_up  0              12.07
          1               7.84
original  0              12.51
          1              12.31
Name: PRICE, dtype: float64 
 

AVG Amount of Purchase by Test Groups And Platforms 
 
 PLATFORM  AB_ID   
Android   level_up   10.77
          original   13.69
iOS       level_up   12.05
          original   11.89
Name: PRICE, dtype: float64 
 

AVG Amount of Purchase by Test Groups And Platforms 
 
 ISNEWACCOUNT  PLATFORM  AB_ID   
0             Android   level_up   11.24
                        original   13.22
              iOS       level_up   12.40
   

In [72]:
df.groupby(['ISNEWACCOUNT','PLATFORM','AB_ID'])['PRICE'].sum()

ISNEWACCOUNT  PLATFORM  AB_ID   
0             Android   level_up    7,555.33
                        original   38,578.02
              iOS       level_up   20,967.28
                        original   72,523.93
1             Android   level_up      448.40
                        original    6,522.50
              iOS       level_up    1,409.52
                        original    6,524.13
Name: PRICE, dtype: float64

In [70]:
df.groupby(['ISNEWACCOUNT','PLATFORM','AB_ID'])['PRICE'].count()

ISNEWACCOUNT  PLATFORM  AB_ID   
0             Android   level_up     672
                        original    2919
              iOS       level_up    1691
                        original    5962
1             Android   level_up      71
                        original     375
              iOS       level_up     166
                        original     685
Name: PRICE, dtype: int64

In [79]:
df.groupby(['ISNEWACCOUNT','PLATFORM','AB_ID'], as_index=False)['PRICE'].mean()


,ISNEWACCOUNT,PLATFORM,AB_ID,PRICE
0,0,Android,level_up,11.24
1,0,Android,original,13.22
2,0,iOS,level_up,12.40
3,0,iOS,original,12.16
4,1,Android,level_up,6.32
5,1,Android,original,17.39
6,1,iOS,level_up,8.49
7,1,iOS,original,9.52


In [148]:
ab_summary[''] = df.pivot_table(values='PRICE', index='AB_ID', aggfunc=np.sum)
# add additional columns to the pivot table
ab_summary['nb_purchase'] = df.pivot_table(values='PRICE', index='AB_ID', aggfunc=lambda x: len(x))
ab_summary['avg_purchase'] = df.pivot_table(values='PRICE', index='AB_ID')

print(ab_summary)

              PRICE  nb_purchase  avg_purchase
AB_ID                                         
level_up  30,380.53     2,600.00         11.68
original 124,148.58     9,941.00         12.49


In [147]:
from scipy.stats import ttest_ind

df_1 = df.loc[ (df['PLATFORM'] == 'iOS') & (df['ISNEWACCOUNT']=='0')]

df_2 = df.loc[ (df['PLATFORM'] == 'iOS') & (df['ISNEWACCOUNT']=='1')]

df_3 = df.loc[ (df['PLATFORM'] == 'Android') & (df['ISNEWACCOUNT']=='0')]

df_4 = df.loc[ (df['PLATFORM'] == 'Android') & (df['ISNEWACCOUNT']=='1')]


c1 = df_1[df_1['AB_ID']=='level_up']
c2 = df_1[df_1['AB_ID']=='original']
print( ttest_ind(c1['PRICE'], c2['PRICE']) )

c1 = df_2[df_2['AB_ID']=='level_up']
c2 = df_2[df_2['AB_ID']=='original']
print(ttest_ind(c1['PRICE'], c2['PRICE']) )

c1 = df_3[df_3['AB_ID']=='level_up']
c2 = df_3[df_3['AB_ID']=='original']
print(ttest_ind(c1['PRICE'], c2['PRICE']) )

c1 = df_4[df_4['AB_ID']=='level_up']
c2 = df_4[df_4['AB_ID']=='original']
print(ttest_ind(c1['PRICE'], c2['PRICE']) )



Ttest_indResult(statistic=0.5109744684525505, pvalue=0.6093836783614459)
Ttest_indResult(statistic=-0.7728734988728705, pvalue=0.4398123608209308)
Ttest_indResult(statistic=-2.5896356369782745, pvalue=0.009646581561604464)
Ttest_indResult(statistic=-3.8959850510227922, pvalue=0.00011286755006567315)


In [130]:
  df_1= df.loc[ df['ISNEWACCOUNT']=='0'] 
  df_3= df.loc[(df['PLATFORM'] == 'Android') & (df['ISNEWACCOUNT']=='0') ]

  cat1 = df_2[df_2['AB_ID']=='level_up']
  cat2 = df_2[df_2['AB_ID']=='original']
  ttest_ind(cat1['PRICE'], cat2['PRICE'])
    
  df_3  


,AB_ID,USER_ID,PRICE,PRODUCT,ISNEWACCOUNT,PLATFORM,COUNTRY
0,original,1700244631376,4.99,com.ludia.jurassicworld.specialoffer_fto_1,0,Android,United States
1,level_up,1700272188399,9.99,com.ludia.jurassicworld.memauto1,0,Android,United States
2,original,1700271842657,74.99,com.ludia.jurassicworld.cardgeneralpack75,0,Android,United States
4,original,1700234432367,9.99,com.ludia.jurassicworld.memauto1,0,Android,United States
7,original,1700282629149,24.99,com.ludia.jurassicworld.cardsuperrarepack,0,Android,United States
10,original,1700283038294,7.99,com.ludia.jurassicworld.specialoffer_fto_2,0,Android,United States
12,level_up,1700284668723,1.99,com.ludia.jurassicworld.hc_starter,0,Android,United States
14,level_up,166436475,14.99,com.ludia.jurassicworld.levelupoffer_3,0,Android,United States
15,original,1700277511080,24.99,com.ludia.jurassicworld.hc_large,0,Android,United States
18,original,174066697,1.99,com.ludia.jurassicworld.hc_starter,0,Android,United States


In [93]:
n=N_A+N_B
bcr=12.49
d_hat=11.68-12.49
abplot(n,bcr,d_hat)

NameError: name 'abplot' is not defined